# Predator-Prey Simulation

In [ ]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import pandas as pd
import plotly.express as px
import random
import warnings

random.seed(1)
warnings.filterwarnings('ignore', category=FutureWarning)

class Rabbit(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
    
    def set_pos(self, pos):
        self.pos = pos
        print(f"{self.unique_id} at {self.pos}")
    
    def move(self):
        try:
            possible_moves = self.model.grid.get_neighborhood(
                self.pos,
                moore=True,
                include_center=False
            )
            new_position = self.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            print(f"{self.unique_id} moved to {self.pos}")
        
        except Exception:
            print(f"{self.unique_id} could not move from {self.pos}")
    
    def step(self):
        self.move()

class Fox(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.energy = 10
    
    def set_pos(self, pos):
        self.pos = pos
        print(f"Fox{self.unique_id} at {self.pos} with energy {self.energy}")
    
    def move(self):
        try:
            possible_moves = self.model.grid.get_neighborhood(
                self.pos,
                moore=True,
                include_center=False
            )
            new_position = self.random.choice(possible_moves)
            if len(self.model.grid[new_position]) != 0:
                for agent in self.model.grid[new_position]:
                    if isinstance(agent, Rabbit):
                        prey = agent
                        print(f"{self.unique_id} at {self.pos} killed {prey.unique_id} at {new_position}")
                        self.model.grid.remove_agent(prey)
                        self.model.schedule.remove(prey)
                        self.energy += 10
                        break
                    else:
                        pass
            
            self.model.grid.move_agent(self, new_position)
            print(f"Fox{self.unique_id} moved to {self.pos} with energy {self.energy}")
            self.energy -= 1
            if self.energy <= 0:
                print(f"{self.unique_id} died at {self.pos}")
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)

        except Exception:
            print(f"{self.unique_id} could not move from {self.pos}")
    
    def step(self):
        self.move()

class Environment(MultiGrid):
    def __init__(self, width, height):
        super().__init__(width, height, torus=False)
    
    def place_agent(self, agent, pos):
        super().place_agent(agent, pos)
    
    def move_agent(self, agent, pos):
        super().move_agent(agent, pos)

    def remove_agent(self, agent):
        super().remove_agent(agent)

class PredatorPrey(Model):
    def __init__(self, width, height, num_rabbits, num_foxes):
        self.schedule = RandomActivation(self)
        self.grid = Environment(width, height)
        self.num_rabbits = num_rabbits
        self.num_foxes = num_foxes
        self._steps = 0
        self._time = 0
        self.dc = DataCollector(agent_reporters={
                "x": lambda a: a.pos[0],
                "y": lambda a: a.pos[1],
                "AgentType": lambda a: a.__class__.__name__
            }
        )
        self.create_agents()

    def create_agents(self):
        for id in range(self.num_rabbits):
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            rabbit = Rabbit(f"Rabbit{id}", self)
            rabbit.set_pos((x, y))
            self.grid.place_agent(rabbit, (x, y))
            self.schedule.add(rabbit)
        
        for id in range(self.num_foxes):
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            fox = Fox(f"Fox{id}", self)
            fox.set_pos((x, y))
            self.grid.place_agent(fox, (x, y))
            self.schedule.add(fox)
        self.dc.collect(self)
    
    def plot_agents_history(self):
        fig = px.scatter(self.dc.get_agent_vars_dataframe().reset_index(),
            x="x", y="y", animation_frame="Step",
            color="AgentType", hover_name="AgentID", animation_group="AgentID",
            range_x=[0,self.grid.width-1], range_y=[0,self.grid.height-1],
            title="Predator-Prey", width=600, height=600
        )
        fig.update_layout(
            xaxis=dict(
                dtick=1,
            ),
            yaxis=dict(
                dtick=1,
            )
        )
        fig.show()
        fig.write_html("./mas.html")

    def step(self):
        self.schedule.step()
        self.dc.collect(self)

print(f"################################## Initialize ##################################")
model = PredatorPrey(10, 10, 12, 12)

for i in range(30):
    print(f"################################## Step {i + 1} ##################################")
    model.step()

model.plot_agents_history()